In [ ]:
!pip install -qU langchain langchain-community langgraph langchain-openai tavily-python langgraph-checkpoint-sqlite

# Build an Agent

Language models by themselves cannot take actions - they just output text. A big use case for LangChain is creating **agents**. Agents are systems that use LLMs as reasoning engines to determine which actions to take and the inputs to pass them. After executing actions, the results can be fed back into the LLM to determine whether mroe actions are needed, or whether it is okay to finish.

In this section, we will build an agent that can interact with a search engine.

## Setup

In [ ]:
anthropic_api_key = 'YOUR_ANTHROPOPIC_API_KEY' # Replace with your actual API key

In [ ]:
import os

langchain_api_key = 'your_langchain_api_key_here'  # Replace with your actual LangChain API key
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_API_KEY'] = langchain_api_key

tavily_api_key = 'your_tavily_api_key_here'  # Replace with your actual Tavily API key
os.environ['TAVILY_API_KEY'] = tavily_api_key

openai_api_key = 'your_openai_api_key_here'  # Replace with your actual OpenAI API key
os.environ['OPENAI_API_KEY'] = openai_api_key

## Define tools

First we need to create the tools we want to use. Our main tool of choice will be Tavily - a search engine. We have a built-in tool in LangChain to easily use Tavily search engine as tool.

In [6]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=5)
search_results = search.invoke('what is the weather in Houston?')
print(search_results)

# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'Houston', 'region': 'Texas', 'country': 'United States of America', 'lat': 29.7631, 'lon': -95.3631, 'tz_id': 'America/Chicago', 'localtime_epoch': 1730141507, 'localtime': '2024-10-28 13:51'}, 'current': {'last_updated_epoch': 1730141100, 'last_updated': '2024-10-28 13:45', 'temp_c': 29.0, 'temp_f': 84.2, 'is_day': 1, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png', 'code': 1003}, 'wind_mph': 11.6, 'wind_kph': 18.7, 'wind_degree': 150, 'wind_dir': 'SSE', 'pressure_mb': 1019.0, 'pressure_in': 30.09, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 52, 'cloud': 25, 'feelslike_c': 32.0, 'feelslike_f': 89.7, 'windchill_c': 28.2, 'windchill_f': 82.8, 'heatindex_c': 30.7, 'heatindex_f': 87.2, 'dewpoint_c': 21.1, 'dewpoint_f': 70.0, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 6.0, 'gust_mph': 13.4, 'gust_kph': 21.5}}"}, {'url': 'https://world-weather.info/forecast/usa/houston

## Using language models

Next, we need to use a language model by to call tools. LangChain supports many different language models that we want interchangably select the one we want to use. Here we use OpenAI as an example

In [9]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model='gpt-4')

We can call the language model by passing in a list of messages. By default, the response is a `content` string.

In [10]:
from langchain_core.messages import HumanMessage

response = model.invoke([HumanMessage(content='Hi!')])
response.content

'Hello! How can I assist you today?'

In order to enable this model to do tool calling, we use `.bind_tools` method to give the language model knowledge of these tools

In [11]:
model_with_tools = model.bind_tools(tools)

We can now call the model.

We first call it with a normal message, and see how it responds.

In [13]:
response = model_with_tools.invoke([HumanMessage(content='Hi!')])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: Hello! How can I assist you today?
ToolCalls: []


Now we call it with some input that would expect a tool to be called

In [14]:
response = model_with_tools.invoke(
    [HumanMessage(content='What is the weather in Houston?')]
)

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Houston'}, 'id': 'call_9mqBDsv2XhAwQxXl1u0QZs5l', 'type': 'tool_call'}]


We can see that there is now NO text content, but there is a tool call. It wants use to call the Tavily Search tool.

In order to actually call the tool, we will want to create our agent.

## Create the agent

We will be using LangGraph to construct the agent.

Now we can intialize the agent with the LLM and the tools. Note that we are passing in the `model`, not `model_with_tools`, because `create_react_agent` will call `.bind_tools` for us under the hood.

In [16]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

## Run the agent

For now, these are all **stateless** queries (it will NOT remember previous interactions). The agent will return the **final** state at the end of the interaction.

First, when there is no need to call a tool:

In [17]:
response = agent_executor.invoke(
    {'messages': [HumanMessage(content='Hi!')]}
)

response['messages']

[HumanMessage(content='Hi!', additional_kwargs={}, response_metadata={}, id='74b9b3b6-874b-41fe-a3f4-0a1819f1e5b0'),
 AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 83, 'total_tokens': 93, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-baa62bd8-f540-4e8d-bd6d-118772c77d79-0', usage_metadata={'input_tokens': 83, 'output_tokens': 10, 'total_tokens': 93, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})]

When the model should be invoking the tool:

In [18]:
response = agent_executor.invoke(
    {'messages': [HumanMessage(content='What is the weather in Houston?')]}
)

response['messages']

[HumanMessage(content='What is the weather in Houston?', additional_kwargs={}, response_metadata={}, id='3e34f218-f158-4f00-af50-71ca1975b00e'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_aJBKcKcE4GdDmVbShpMqvwOH', 'function': {'arguments': '{\n  "query": "current weather in Houston"\n}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 88, 'total_tokens': 110, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-ca717ecc-7e6f-4767-abd7-77aec2dcfa87-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Houston'}, 'id': 'call_aJBKcKcE4GdDmVbShpMqvwOH', 'type': 'tool_call'}], usage_metadata={'input_tokens': 88, 'ou

## Streaming messages

If the agent is executing multiple steps, that may take a while. In order to show intermediate progress, we can stream back messages as they occur.

In [19]:
for chunk in agent_executor.stream(
    {'messages': [HumanMessage(content='What is the weather in Houston?')]}
):
    print(chunk)
    print('-'*20)

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_3V88EJuFQwAPHuwnwTMj1l5z', 'function': {'arguments': '{\n  "query": "Current weather in Houston"\n}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 88, 'total_tokens': 110, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-0c1a3933-cfce-4769-8f50-626264d9f6d7-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'Current weather in Houston'}, 'id': 'call_3V88EJuFQwAPHuwnwTMj1l5z', 'type': 'tool_call'}], usage_metadata={'input_tokens': 88, 'output_tokens': 22, 'total_tokens': 110, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}}

## Streaming tokens

In addition to streaming back messages, it is also useful to be streaming back tokens, with the `.astream_events` method.

In [20]:
async for event in agent_executor.astream_events(
    {'messages': [HumanMessage(content='What is the weather in Houston?')]},
    version='v1',
):
    kind = event['event']

    if kind == 'on_chain_start':
        if (
            event['name'] == 'Agent'
        ): # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print(
                f"Starting agent: {event['name']} with input: {event['data'].get('input')}"
            )
    elif kind == 'on_chain_end':
        if (
            event['name'] == 'Agent'
        ): # Was assgined when creating the agent with `.with_config({"run_name": "Agent"})`
            print()
            print('--')
            print(
                f"Done agent: {event['name']} with output: {event['data'].get('output')['output']}"
            )

    if kind == 'on_chat_model_stream':
        content = event['data']['chunk'].content
        if content:
            # Empty content in the context of OpenAI means that
            # the model is asking for a tool to be invoked.
            # so we only print non-empty content
            print(content, end='|')
    elif kind == 'on_tool_start':
        print('--')
        print(
            f"Starting tool: {event['name']} with inputs: {event['data'].get('input')}"
        )
    elif kind == 'on_tool_end':
        print(f"Done tool: {event['name']}")
        print(f"Tool output was {event['data'].get('output')}")
        print('--')

--
Starting tool: tavily_search_results_json with inputs: {'query': 'current weather in Houston'}
Done tool: tavily_search_results_json
Tool output was content='[{"url": "https://www.weatherapi.com/", "content": "{\'location\': {\'name\': \'Houston\', \'region\': \'Texas\', \'country\': \'United States of America\', \'lat\': 29.7631, \'lon\': -95.3631, \'tz_id\': \'America/Chicago\', \'localtime_epoch\': 1730142981, \'localtime\': \'2024-10-28 14:16\'}, \'current\': {\'last_updated_epoch\': 1730142900, \'last_updated\': \'2024-10-28 14:15\', \'temp_c\': 29.4, \'temp_f\': 84.9, \'is_day\': 1, \'condition\': {\'text\': \'Partly cloudy\', \'icon\': \'//cdn.weatherapi.com/weather/64x64/day/116.png\', \'code\': 1003}, \'wind_mph\': 12.3, \'wind_kph\': 19.8, \'wind_degree\': 147, \'wind_dir\': \'SSE\', \'pressure_mb\': 1018.0, \'pressure_in\': 30.07, \'precip_mm\': 0.0, \'precip_in\': 0.0, \'humidity\': 52, \'cloud\': 25, \'feelslike_c\': 32.4, \'feelslike_f\': 90.3, \'windchill_c\': 28.2, \

## Adding in memory

As mentioned earlier, this agent is stateless, which means that it does not remember previous interaction. To give it memory we need to pass in a checkpointer. When passing in a checkpointer, we also have to pass in a `thread_id` when invoking the agent (so it knows which thread/conversation to resume from).

In [21]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

In [22]:
agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {'configurable': {'thread_id': 'abc123'}}

In [23]:
for chunk in agent_executor.stream(
    {'messages': [HumanMessage(content='Hi! My name is Bin!')]}, config
):
    print(chunk)
    print('------')

{'agent': {'messages': [AIMessage(content='Hello, Bin! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 88, 'total_tokens': 100, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-14df8061-2ac1-41c2-9dde-8a68ae8ae5d4-0', usage_metadata={'input_tokens': 88, 'output_tokens': 12, 'total_tokens': 100, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})]}}
------


In [24]:
for chunk in agent_executor.stream(
    {'messages': [HumanMessage(content='What is my name?')]}, config
):
    print(chunk)
    print('------')

{'agent': {'messages': [AIMessage(content='Your name is Bin. How can I assist you further?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 112, 'total_tokens': 125, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-11593860-1d89-4115-b46d-83546a768d02-0', usage_metadata={'input_tokens': 112, 'output_tokens': 13, 'total_tokens': 125, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})]}}
------


If we want to start a new conversation, all we have to do is change the `thread_id` used in the previous call:

In [25]:
config2 = {'configurable': {'thread_id': 'xyz789'}}

for chunk in agent_executor.stream(
    {'messages': [HumanMessage(content='What is my name?')]}, config2
):
    print(chunk)
    print('------')

{'agent': {'messages': [AIMessage(content="As an AI, I don't have access to personal data about individuals unless it has been shared with me in the course of our conversation. I am designed to respect user privacy and confidentiality.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 39, 'prompt_tokens': 86, 'total_tokens': 125, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-0ae66524-8d87-4170-890e-24ec6f9137c1-0', usage_metadata={'input_tokens': 86, 'output_tokens': 39, 'total_tokens': 125, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})]}}
------
